In [1]:
import numpy as np
import numpy.linalg as la
import cvxpy as cp

from functools import partial

  
import matplotlib.pyplot as plt
import labellines as ll


from shapely.geometry import Point
from descartes import PolygonPatch

from RKHS import feature

In [2]:
# The differentiable kernel function with parameters c,l not filled.
def h(r,c,l):
    return c * np.exp(-(r**2) / l**2)

def k(x1,x2,c,l):
    small_sig = 1e-10 # This is needed for numerical stability.
    return h(np.linalg.norm(x1-x2+small_sig,axis = -1),c,l)

def Gram(kernel,x):
    KA = kernel(x[:,np.newaxis,:],x)
    return KA

In [3]:
def h_inv(y,c,l):
    return np.sqrt(np.log(c/y))*l

In [4]:
kernel = partial(k,c=1,l=1)

# Set up the reference points

In [27]:
nx, ny = (10, 10)
x = np.linspace(-2, 4, nx)
y = np.linspace(-2, 4, ny)
xv, yv = np.meshgrid(x, y)

ref = np.array([xv.ravel(),yv.ravel()]).T

In [28]:
ref[33]

array([0., 0.])

In [29]:
radius = 2
Rs = np.zeroes(len(ref))
Rs[33] = radius

K = Gram(kernel,ref)

l,U =la.eig(K)

# M = np.diag(1/np.sqrt(l)).dot(U) # M.T.dot(M) = K^{-1} is satisfied.

# Another idea of choosing M, not from the eigen decomp approach though.
M = la.cholesky(la.inv(K)).T

AttributeError: module 'numpy' has no attribute 'zeroes'

# Set up the true x locations

In [6]:
x_true = np.array([[0,0],[0,1],[1,1],[2,1],[2,2],[2,3],[3,3],[4,3],[4,4]])
S = Gram(kernel,x_true)

# Check if the reference points are dense enough

In [7]:
feat_true = feature(kernel, ref,x_true)

In [8]:
feat_true.T.dot(feat_true)-S

array([[ 1.68153595e-08,  1.04072435e-08,  5.13740250e-09,
         6.82572878e-10,  1.49657482e-10,  1.19323885e-11,
         5.24114256e-13,  9.85065184e-15,  8.70360847e-16],
       [ 1.05543952e-08, -2.83141652e-03, -1.04161735e-03,
        -5.18583435e-05,  6.82638229e-10,  3.46903494e-05,
         2.33746782e-07,  2.13242577e-10,  8.09093232e-15],
       [ 5.24567079e-09, -1.04161720e-03, -5.65483291e-03,
        -1.04161735e-03,  5.13925055e-09,  6.96772486e-04,
         7.29678647e-05,  2.33745653e-07,  3.88428558e-13],
       [ 6.90658409e-10, -5.18583289e-05, -1.04161720e-03,
        -2.83141652e-03,  1.04097740e-08,  1.89402332e-03,
         6.96772368e-04,  3.46903104e-05,  7.58609603e-12],
       [ 1.50194223e-10,  6.90723760e-10,  5.24751884e-09,
         1.05569258e-08,  1.68157208e-08,  9.21904925e-09,
         4.06157422e-09,  4.43460672e-10,  8.08425322e-11],
       [ 1.19369092e-11,  3.46903499e-05,  6.96772494e-04,
         1.89402333e-03,  9.36620104e-09, -4.256330

# Define some constants

In [9]:
N = len(ref)
T = len(x_true)

# Define the decision variables that represent the feature matrix, solve for an feasible solution.

In [18]:
H = h(Rs,c=1,l=1).reshape(-1,1)

H.dot(np.ones((1,T))).shape

phi = cp.Variable((N,T))

constraints = [la.inv(M) @ phi>= H.dot(np.ones((1,T)))]
# constraints = [la.inv(M) @ phi>= 0]


prob=cp.Problem(cp.Maximize(0),constraints)

prob.solve()

phi0 = phi.value

phi_prev=phi0

# Perform the alternating solver

In [19]:
for _ in range(200):
    prob = cp.Problem(cp.Minimize(cp.norm(S-phi.T @ phi_prev)**2+0.1*cp.norm(phi-phi_prev)**2),constraints)
    prob.solve()
    phi_prev = phi.value
    
    print(prob.value,)
    if prob.value<1e-7:
        break

12.257599307765712
2.7800179576452226
2.7749850566784895
2.774980582599864
2.7735922684921723
2.3947824680149723
0.18747243585569986
0.1404259090683563
0.10440314920642543
0.0811060225254309
0.06212602898349043
0.04894891448908617
0.0381287794746278
0.030259645386290758
0.023794788318793333
0.018963422043247857
0.015001787208252213
0.011993716291199785
0.009527261390604391
0.007624267894464974
0.006075588408105649
0.0048682974627762094
0.003886756514197714
0.00311705665378324
0.002492507001049377
0.0019996933586407177
0.001601581019264396
0.0012858057218557498
0.0010304205881524252
0.0008279076847130846
0.0006641254456764195
0.0005338262762833056
0.0004279964186469395
0.00034376427112117346
0.00027616723930774627
0.00022228340736786176
0.0001789031125078275
0.00014379356295787695
0.00011588474748873012
9.691892285223057e-05
7.415711167069815e-05
5.92835390208662e-05
4.749641807537969e-05
3.826066484877966e-05
3.0765594956624604e-05
2.4764964524635022e-05
1.9934753978385862e-05
1.650057

# The solution $\phi$ is good enough using alternating method.

# We next apply brute force random search to recover all $x_i$.
       
$$x_i = \arg\min_{x\in{\{v_{1:1e5}\}\sim Unif(Search Region)}} ||\phi_i - M k(a_{1:N},x)||$$

In [34]:
phi_prev

array([[-8.36184748e-02, -7.29107098e-02, -5.99496208e-02,
        -1.16424520e-01, -2.34236136e-02, -1.80693150e-01,
         1.59481872e-01,  2.47098258e-01,  2.77319481e-01],
       [ 4.69756433e-02, -7.28722358e-02,  8.32479711e-02,
         1.59059029e-01,  2.67610500e-03, -1.35817657e-01,
         1.51202242e-01,  3.15276532e-02, -6.36822274e-02],
       [-5.48169867e-02,  1.88321142e-01,  1.52158790e-01,
        -3.54733721e-02,  5.86659149e-02,  7.51849828e-02,
        -1.15488109e-01, -2.98892913e-02, -4.09271460e-02],
       [ 9.79187110e-02,  1.10124830e-01,  8.49782940e-02,
         3.33666791e-03,  6.04797987e-02,  1.33393076e-01,
        -5.54889132e-02, -7.32834359e-03, -1.39146874e-01],
       [ 9.07142861e-02,  2.83491264e-02, -6.77140167e-03,
         6.65704803e-03,  9.89681096e-02, -7.42617078e-02,
         6.86464410e-02, -4.18240717e-03,  8.54039057e-02],
       [ 1.06950081e-03,  8.03679951e-02, -3.44131803e-02,
        -1.62020764e-02,  1.03330227e-01,  5.004258

In [35]:
feat_true

array([[-3.65199762e-16,  2.59858322e-13,  1.30510420e-04,
        -6.40344440e-14, -1.00624052e-17,  1.96146549e-14,
         1.22315830e-05, -1.12563122e-14, -7.58941521e-19],
       [ 2.25141603e-13, -4.85784965e-13, -2.13002541e-04,
         1.01123974e-13,  3.40484093e-17, -3.09521833e-14,
        -1.91326912e-05,  1.75688268e-14,  1.08420217e-18],
       [ 3.70877729e-12,  1.31483038e-12,  3.72427428e-04,
        -1.62068829e-13,  2.98790981e-17,  4.96581447e-14,
         3.00539174e-05, -2.74415857e-14, -2.81892565e-18],
       [ 3.15460815e-11,  7.53509237e-03, -7.87978796e-04,
         2.71806108e-13,  2.21261316e-16, -8.31128166e-14,
        -4.76999097e-05,  4.29678731e-14,  3.90312782e-18],
       [ 4.12434116e-11,  8.18437283e-03,  3.41670289e-03,
        -5.04483534e-13,  2.16171225e-16,  1.55567496e-13,
         7.76746125e-05, -6.76364794e-14, -7.80625564e-18],
       [ 1.84285071e-11,  2.58087423e-03,  9.46598591e-03,
         1.37139655e-12,  1.10850539e-16, -4.094247

In [47]:
x_min = 0
x_max = 10
n_cand = 100000
cand_x = np.random.uniform(x_min,x_max,size = [n_cand,2])

In [48]:
cand_feat = feature(kernel, ref, cand_x)

In [67]:
# dist=la.norm(cand_feat[:,np.newaxis,:]-phi_prev[:,:,np.newaxis],axis=0)

# best_indx = np.argmin(dist,axis=1)

# best_fit = cand_x[best_indx]

# best_fit

# dist.shape

# best_indx

# dist[np.arange(9),best_indx]

In [56]:
dist = la.norm(cand_feat-phi_prev[:,0].reshape(-1,1),axis = 0)

cand_x[np.argmin(dist)]

np.min(dist)

0.9625311839830882

Such loss is not good enough at all. This pretty much means the $\phi$ returned by alternating algorithm is NOT admissible.

In [57]:
dist = la.norm(cand_feat-feat_true[:,0].reshape(-1,1),axis = 0)

cand_x[np.argmin(dist)]

np.min(dist)

0.020991120418440964

In comparison, the feature vector of $x_{true}$ is clearly an admissible one.